In [91]:
#!pip install --upgrade oct2py
import numpy as np
from oct2py import octave
import matplotlib.pyplot as plt
import os
import pandas as pd
octave.addpath('C:/dynare/6.1/matlab')  
#octave.addpath('/opt/homebrew/opt/dynare/lib/dynare/matlab')

'.;C:\\dynare\\6.1\\matlab\\dseries\\src\\mdbnomics2dseries;C:\\dynare\\6.1\\matlab\\dseries\\src\\read;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\is;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\op;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\convert;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\str;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\insert;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\file;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\from;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\get;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\print;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\variables;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\cumulate;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\struct;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\misc;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\x13;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\dplot;C:\\dynare\\6.1\\contrib\\ms-sbvar\\TZcode\\MatlabFiles;C:\\

In [92]:
n_goods=3

In [93]:
#from MyPy import DyPy

In [94]:
with open('t_call.m', 'w') as file:
    file.write("stoch_simul(nograph, order=1, periods=10000, irf=100);")

In [95]:
%run make_model.ipynb
%run make_vars.ipynb
%run make_parameters_and_shocks.ipynb
%run make_steady_state.ipynb


Makes a call to Octave, which makes a call to Dynare. Then the `.mod` file is run in Dynare. Output from Dynare is displayed as it runs.

In [96]:
octave.dynare('model_1.mod')
oo = octave.pull('oo_')
mm = octave.pull('M_')

Starting Dynare (version 6.1).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 12 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the static model...
7 block(s) found:
  6 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 6 equation(s)
                                 and 6 feedback variable(s).
Computing dynamic model derivatives (order 1).
Normalizing the dynamic model...
Finding the optimal block decomposition of the dynamic model...
2 block(s) found:
  1 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 6 equation(s)
                                 and 6 feedback variable(s).
Preprocessing completed.
Preprocessing time: 0h00m00s.

STEADY-STATE RESULTS:

c      		 0.186403
i      		 0.358507
k      		 1.27678
y      		 0.446873
n0     		 -0.721121
s0     		 -0.7211

In [97]:
n_goods

3

In [98]:
names1 = [attr for attr in dir(oo) if not attr.startswith('__')]

Now we want to create a class where each each attribute in oo becomes an attribute of this class. So far we take only one item from mm. This is a list of endogenous variables. Below we define the class. 

In [99]:
class from_octave:
	def __init__(self, oo_structure, mm_structure, attribute_names):
		self.oo_structure = oo_structure
		self.mm_structure = mm_structure
		self. attribute_names =  attribute_names
		self.create_attributes()
		self.make_list()
     
	def create_attributes(self): 
		for attr in self.attribute_names:
			setattr(self, attr, getattr(self.oo_structure, attr)) #this is the attributes we pull from the oo structure. We are making them attributes of the class
	
	def make_list(self): 
		self.names = list(getattr(self.mm_structure, 'endo_names')) #getting one item from the mm structure. This is the endo_names
		self.names = [item[0] for item in self.names] #pulling mm gives us a list of cell arrays. We are pulling the first item from each cell array

We see that we now can reference any of the data from `oo` with the `tt.` notation. Some examples are below. 

In [100]:
tt = from_octave(oo, mm, names1)

We will care about the irfs but these are also of type `oct2py.io.Struct`. We convert it as above. We will likley make this part of the above class later, but for now we create a new class. 

In [101]:
names2 = [attr for attr in dir(tt.irfs) if attr.endswith('shock')]
#names2 = [x for _, x in sorted(zip(tt.names, names2), key=lambda pair: pair[0][0])]

I will put more desciptors here later. However, it is getting the impulse responses as difference and levels. It is simultanosuly creating two dataframes containing all differences and levels data.  

In [102]:
#recall make_levels is created in simpler ffolder
import numpy as np
import pandas as np
class make_irfs:
	def __init__(self, dta, vars, names):
		self.data = dta
		self.vars = vars
		self.names = names
		self.create_difs()
		
  	
	def create_difs(self):
		for var in self.vars:
			setattr(self, var, pd.DataFrame())  # Initialize an empty DataFrame with the dynamic name
			list_short = [name for name in self.names if name.startswith(var)]
			for attr in list_short:
				x = getattr(self.data, attr)
				x = x.flatten()
				getattr(self, var)[attr] = x  # Assign the flattened array to the DataFrame
				new_columns = [col.replace(var + '_', '').replace('shock', '') for col in getattr(self, var).columns]
				getattr(self, var).columns = new_columns
    
    



In [103]:
varlist=['c','i','y','k','n0','s0']

In [104]:
ir = make_irfs(tt.irfs,varlist, names2)
print(ir.__dict__.keys())

dict_keys(['data', 'vars', 'names', 'c', 'i', 'y', 'k', 'n0', 's0'])


Lets get a sense of what we have. 

This will also need more documentation, but it is getting the results of dynare's simulation. It defines the individual series and also puts all in a dataframe. 

In [105]:
import numpy as np

class make_data:
	def __init__(self, dta,names):
		self.data = dta
		self.names = names
		self.make_levels()
		self.make_raw()

	def make_raw(self):
		self.df_raw = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			new_attr =attr + 'r'
			raw_level = self.data[j]
			setattr(self, new_attr, raw_level)
			self.df_raw[new_attr] = raw_level
			
  		
	def make_levels(self):
		self.df = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			raw_level = self.data[j]
			if attr != 'i' and 'theta' not in attr:
				current_level=np.exp(raw_level)
			else:
				current_level=raw_level
			setattr(self, attr, current_level)
			self.df[attr] = current_level  # Store in DataFrame



In [106]:
eg=make_data(tt.endo_simul,tt.names)

In [107]:
def x_find(g,tt):
    thetag='theta'+str(g)
    ag='a'+str(g)
    omegag='omega'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    ai=eg.df[ag][tt]
    a0=eg.df['a0'][tt]
    k=eg.df['k'][tt-1]
    s0=eg.df['s0'][tt]
    n0=eg.df['n0'][tt]
    theta0=eg.df['theta0'][tt]
    eps=params['eps']
    omegai=params[omegag]
    omega0=params['omega0']
    
    x=(k*s0/n0)**((theta0-thetai)*(1-eps))*(omegai/omega0)**eps* \
    ((a0)**(1-theta0)/(ai)**(1-thetai)*(theta0/thetai)**thetai*((1-theta0)/(1-thetai))**(1-thetai))**(1-eps)
    
    return x

In [108]:
def X_find(t):
    X=0
    for g in range(0,n_goods):
        X+=x_find(g,t)
    return X

In [109]:
def Z_find(t):
    X=0
    for g in range(0,n_goods):
        thetag='theta'+str(g)
        thetai=eg.df[thetag][t]
        X+=x_find(g,t)*thetai
    return X

In [110]:
eg.df['X'] = [X_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]
eg.df['Z'] = [Z_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]
eg.df['Q']=(eg.df['Z']-eg.df['theta0']*eg.df['X'])/eg.df['X']

In [111]:
def py_n(tt,g):
    
    thetag='theta'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    a0=eg.df['a0'][tt]
    s0=eg.df['s0'][tt]
    n0=eg.df['n0'][tt]
    k=eg.df['k'][tt-1]
    prod=(1-theta0)*a0**(1-theta0)*k**theta0*(s0/n0)**theta0/(1-thetai)
    return prod

In [112]:
def n_i(tt,g):
    
    thetag='theta'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    y=eg.df['y'][tt]
    c=eg.df['c'][tt]
    X=eg.df['X'][tt]
    Q=eg.df['Q'][tt]
    
    xi=x_find(g,tt)
    if g>0:
        ni=xi/X*c/y*(1-thetai)/(1-theta0-c/y*Q)
    else:
        ni=(1-theta0)*(1-c/y*(1-xi/X))/(1-theta0-c/y*Q)
    return ni

In [113]:

def within_find(tt):
    within=0
    ylag=eg.df['y'][tt-1]
    a=eg.df['a0'][tt]/eg.df['a0'][tt-1]
    for g in range(0,n_goods):
        withing=(a*py_n(tt,g)-py_n(tt-1,g))*n_i(tt-1,g)
        within+=withing
    return within/ylag

In [114]:

def static_find(tt):
    static=0
    N=0
    N_minus=0
   
    ylag=eg.df['y'][tt-1]
    
    for g in range(0,n_goods):
       
        staticg=(n_i(tt,g)-n_i(tt-1,g))*py_n(tt-1,g)
        
        
        N+=n_i(tt,g)
        N_minus+=n_i(tt-1,g)
        if g==2:
            (print('check1', N,N_minus))
        if g==0:
            (print('check2',n_i(tt,g)-eg.df['n0'][tt]))
        static+=staticg
        
    return static/ylag

In [115]:
def dynamic_find(tt):
    dynamic=0
    ylag=eg.df['y'][tt-1]

    a=eg.df['a0'][tt]/eg.df['a0'][tt-1]
    for g in range(0,n_goods):
        
        dynamicg=(a*py_n(tt,g)-py_n(tt-1,g))*(n_i(tt,g)-n_i(tt-1,g))
        dynamic+=dynamicg
    return dynamic/ylag

In [116]:

eg.df['grow'] = (eg.df['y']*eg.df['a0']/eg.df['a0'].shift(1)-eg.df['y'].shift(1))/eg.df['y'].shift(1)
eg.df.loc[0, 'grow'] = np.nan
eg.df['within'] = [within_find(t) if t > 1 and t < eg.df.shape[0]-1 else np.nan for t in range(eg.df.shape[0])]
eg.df['static'] = [static_find(t) if t > 1 and t < (eg.df.shape[0]-1) else np.nan for t in range(eg.df.shape[0])]
eg.df['dynamic'] = [dynamic_find(t) if t > 1 and t < eg.df.shape[0]-1 else np.nan for t in range(eg.df.shape[0])]
eg.df['check'] = eg.df['grow']-eg.df['within']-eg.df['static']-eg.df['dynamic']


check2 0.0001766732174843244
check1 1.0000000000000004 1.0
check2 0.0005463887082998165
check1 1.0000000000000002 1.0000000000000004
check2 0.00014521923078580912
check1 0.9999999999999998 1.0000000000000002
check2 0.00107251636705874
check1 1.0 0.9999999999999998
check2 -0.00045600194347478196
check1 1.0000000000000002 1.0
check2 0.00015412650985563436
check1 1.0000000000000002 1.0000000000000002
check2 0.000634559581027816
check1 1.0 1.0000000000000002
check2 -0.0009434078324987993
check1 1.0 1.0
check2 -5.216670379792898e-05
check1 1.0 1.0
check2 -0.0006640985816335765
check1 1.0 1.0
check2 -0.002673828923667909
check1 0.9999999999999999 1.0
check2 -0.0002653132456679752
check1 0.9999999999999996 0.9999999999999999
check2 0.0001469024070662739
check1 0.9999999999999999 0.9999999999999996
check2 -0.0001395652293350036
check1 0.9999999999999999 0.9999999999999999
check2 -0.000365528027972295
check1 1.0 0.9999999999999999
check2 -5.981256472842311e-05
check1 1.0 1.0
check2 -0.001000520

In [117]:
eg.df

,c,i,k,y,n0,s0,theta0,theta1,theta2,a0,a1,a2,X,Z,Q,grow,within,static,dynamic,check
0,1.249063,0.409209,3.636134,1.660369,0.486862,0.514504,0.371435,0.336633,0.357333,1.051757,1.059442,1.055584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.246601,0.385749,3.658692,1.633156,0.486823,0.499942,0.360537,0.350744,0.346782,1.039592,1.017201,1.120646,2.997506,1.057251,-0.007827,-0.027767,NaN,NaN,NaN,NaN
2,1.145064,0.324670,3.616963,1.471039,0.489603,0.495120,0.351819,0.351990,0.341724,0.897433,1.085343,1.099863,2.881997,1.004595,-0.003243,-0.222436,-0.222193,0.000023,-0.000069,-0.000197
3,1.089016,0.280910,3.536384,1.376165,0.499352,0.475579,0.336748,0.363416,0.353277,0.839455,1.135322,1.098793,2.796453,0.980425,0.013848,-0.124933,-0.126829,0.000048,-0.000367,0.002214
4,1.028243,0.285765,3.470094,1.320685,0.488423,0.511031,0.354046,0.334766,0.333257,0.770931,1.227985,0.902043,2.833368,0.966340,-0.012989,-0.118654,-0.114822,0.000223,-0.000547,-0.003508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.563634,0.437168,4.037788,2.009799,0.470203,0.465252,0.341865,0.331948,0.361156,1.411090,0.995541,1.221073,3.165715,1.091043,0.002779,0.041423,0.041185,0.000187,-0.000055,0.000106
9996,1.628071,0.476314,4.123348,2.125903,0.463987,0.485620,0.357273,0.332858,0.341332,1.483623,0.948270,1.304600,3.226897,1.107426,-0.014087,0.112140,0.108092,0.000074,-0.000256,0.004229
9997,1.688477,0.493360,4.221928,2.210408,0.460428,0.479841,0.362091,0.305489,0.382119,1.542961,0.843417,1.279372,3.313444,1.149156,-0.015275,0.081335,0.079640,-0.000474,-0.002711,0.004881
9998,1.525488,0.406754,4.209729,1.933916,0.455361,0.460790,0.347410,0.334858,0.349467,1.276641,0.750236,1.165150,3.231143,1.109558,-0.004015,-0.276099,-0.270009,0.001798,-0.001559,-0.006329


In [118]:

eg.df.shape[0]

10000

In [119]:
.5/3


0.16666666666666666

In [120]:

# Function to highlight the diagonal
def highlight_diagonal(data):
	attr = 'background-color: yellow'
	# Create a DataFrame with empty strings
	df_styler = pd.DataFrame('', index=data.index, columns=data.columns)
	# Set the diagonal to the desired attribute
	np.fill_diagonal(df_styler.values, attr)
	return df_styler

# Apply the function to the DataFrame
df_cov = df_cov.style.apply(highlight_diagonal, axis=None)

# Display the styled DataFrame
df_cov

NameError: name 'df_cov' is not defined